# EDA, Pairs and Stripes on Hadoop

## EDA

In [8]:
# Download data
!wget https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0 -O Consumer_Complaints.csv

--2017-01-27 04:28:11--  https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0
Resolving www.dropbox.com... 162.125.65.1
Connecting to www.dropbox.com|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/lTCiJCchrbPy2xfvRCGbPkOLVweAEPNXtByrOgoK90VPHNJbzFX1fD0DvVjf2l5w/file [following]
--2017-01-27 04:28:12--  https://dl.dropboxusercontent.com/content_link/lTCiJCchrbPy2xfvRCGbPkOLVweAEPNXtByrOgoK90VPHNJbzFX1fD0DvVjf2l5w/file
Resolving dl.dropboxusercontent.com... 162.125.65.6
Connecting to dl.dropboxusercontent.com|162.125.65.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50906486 (49M) [text/csv]
Saving to: “Consumer_Complaints.csv”

100%[======================================>] 50,906,486  7.92M/s   in 6.5s    

2017-01-27 04:28:19 (7.43 MB/s) - “Consumer_Complaints.csv” saved [50906486/50906486]



In [9]:
!hdfs dfs -rm  /home/cloudera/dfs/Consumer_Complaints/Consumer_Complaints.csv

Deleted /home/cloudera/dfs/Consumer_Complaints/Consumer_Complaints.csv


In [32]:
# Create HDFS directories
!hdfs dfs -mkdir -p /home/cloudera/dfs/Consumer_Complaints
!hdfs dfs -mkdir -p /home/cloudera/dfs/short-text


In [10]:
# Put the data into HDFS
!hdfs dfs -put Consumer_Complaints.csv /home/cloudera/dfs/Consumer_Complaints
!hdfs dfs -put input.txt /home/cloudera/dfs/short-text

put: `/home/cloudera/dfs/short-text/input.txt': File exists


In [362]:
%%writefile complaintCountsMapper.py
#!/usr/bin/env python


import sys 
from csv import reader

# ------------------------ define function ------------------------------ #

def mapper_runner(csv_file):
    for row in reader(csv_file):

        try:
            # the values in the first (id) column are integers except the header
            is_header = int(row[0])
            # Product is the second column
            print row[1]
        except:
            # if the id is not an integer, it is a header, do not map it
            pass

# -------------------------------run------------------------------------- #
    
if __name__== "__main__":
    mapper_runner(sys.stdin)



Overwriting complaintCountsMapper.py


In [361]:
%%writefile complaintCountsReducer.py
#!/usr/bin/env python

import sys

# ---------------------------- define functions-------------------------- #
def product_counter(product_type):
    '''Emits a Product counter of product types for the jobtracker UI'''
    return sys.stderr.write("reporter:counter:Product,{0},1\n".format(product_type))


def reducer_runner():
     for product in sys.stdin:
            product = product.strip()

            # classify type of product
            if product != 'Debt collection' and product != 'Mortgage':
                product = 'Other'
        # place counter in the jobtracker UI
            sys.stderr.write("reporter:counter:Product,{0},1\n".format(product))


# --------------------------------- run --------------------------------- #        
        
if __name__ == "__main__":
    reducer_runner()
    

Overwriting complaintCountsReducer.py


In [22]:
!chmod a+x complaintCountsReducer.py
!chmod a+x complaintCountsMapper.py

In [363]:
# Hadoop command
!hdfs dfs -rm -r /home/cloudera/W261/Spring2017/HW3/hw3.1-output

!sudo hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar \
    -mapper /home/cloudera/W261/Spring2017/HW3/complaintCountsMapper.py \
    -reducer /home/cloudera/W261/Spring2017/HW3/complaintCountsReducer.py \
    -input /home/cloudera/dfs/Consumer_Complaints/Consumer_Complaints.csv  \
    -output /home/cloudera/W261/Spring2017/HW3/hw3.1-output 


Deleted /home/cloudera/W261/Spring2017/HW3/hw3.1-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar] /tmp/streamjob6538853055586384922.jar tmpDir=null
17/01/29 01:54:35 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 01:54:35 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 01:54:36 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/29 01:54:36 INFO mapreduce.JobSubmitter: number of splits:2
17/01/29 01:54:36 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1484923209747_0225
17/01/29 01:54:37 INFO impl.YarnClientImpl: Submitted application application_1484923209747_0225
17/01/29 01:54:37 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1484923209747_0225/
17/01/29 01:54:37 INFO mapreduce.Job: Running job: job_1484923209747_0225
17/01/29 01:54:44 INFO mapreduce.Job: Job job_1484923209747_0225 running in uber mode : fals

![Product counters](counter_31.png)


__3.2.A__     
Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many times the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.

__3.2.B__   
Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

__3.2.C__     
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

Using a single reducer: 
- What are the top 50 most frequent terms in your word count analysis?    
- Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order.    
- Present bottom 10 tokens (least frequent items). 

In [ ]:
%%writefile reducer3.2.A.py
#!/usr/bin/env python


import sys


# ---------------------- define function ---------------------------- #

def reducer_runner():
    # initialize variables
    cur_key = None
    cur_count = 0

    # loop over STDIN
    for line in sys.stdin:
        # and sum the count of the same consecutive
        key,count = line.lower().strip().split()
    if key == cur_key:
        cur_count += int(count)
    else:
        if cur_key:
            print '{0}\t{1}'.format(cur_key,cur_count)
    
        cur_key = key
        cur_count = int(count)

        
# emit the last key
print '{0}\t{1}'.format(cur_key,cur_count)

# ------------------------------ run -------------------------------- #

if __name__ == "__main__":
    sys.stderr.write("reporter:counter:Wordcount job,Reducer,1\n")
    reducer_runner()


In [37]:
!chmod a+x mapper3.2.A.py
!chmod a+x reducer3.2.A.py

In [364]:
# Hadoop command

!hdfs dfs -rm -r /home/cloudera/W261/Spring2017/HW3/short-text-output

!sudo hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar -D mapreduce.job.maps=1 \
    -D mapreduce.job.reduces=4 \
    -mapper /home/cloudera/W261/Spring2017/HW3/mapper3.2.A.py \
    -reducer /home/cloudera/W261/Spring2017/HW3/reducer3.2.A.py \
    -input /home/cloudera/dfs/short-text/input.txt  \
    -output /home/cloudera/W261/Spring2017/HW3/short-text-output \
    

Deleted /home/cloudera/W261/Spring2017/HW3/short-text-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar] /tmp/streamjob84348425894680986.jar tmpDir=null
17/01/29 01:56:56 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 01:56:56 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 01:56:57 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/29 01:56:57 INFO mapreduce.JobSubmitter: number of splits:1
17/01/29 01:56:57 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1484923209747_0226
17/01/29 01:56:57 INFO impl.YarnClientImpl: Submitted application application_1484923209747_0226
17/01/29 01:56:58 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1484923209747_0226/
17/01/29 01:56:58 INFO mapreduce.Job: Running job: job_1484923209747_0226
17/01/29 01:57:05 INFO mapreduce.Job: Job job_1484923209747_0226 running in uber mode : f

### SCREENSHOT OF JOB TRACKER UI COUNTERS

![](counter_32A.png)

### 3.2.B SOLUTION

In [378]:
%%writefile mapper3.2.B.py
#!/usr/bin/env python

import sys 
import re
from csv import reader

# ------------------------------ define function ---------------------------- #
def mapper_runner():

    WORD_RE = re.compile(r"[\w']+")

    for row in reader(sys.stdin):
        try:
             # the values in the first (id) column are integers except the header
            is_header = int(row[0])
            # Issue is the fourth column
            issue = row[3]
            for word in WORD_RE.findall(issue):
                print '{0}\t1'.format(word)
        except:
            # if the id can't be cast to int, it is a header, do not map it
            pass

# -------------------------------- run ---------------------------------------- # 

if __name__ == "__main__":
    sys.stderr.write("reporter:counter:Wordcount,Mapper,1\n")
    mapper_runner()


Overwriting mapper3.2.B.py


In [379]:
%%writefile reducer3.2.B.py
#!/usr/bin/env python


import sys

# ------------------------------ define functions -------------------------------- #

def reducer_runner():
    
    # define variables
    cur_key = None
    cur_count = 0

    for line in sys.stdin:
        key,count = line.lower().strip().split()
        if key == cur_key:
            cur_count += int(count)
        else:
            if cur_key:
                print '{0}\t{1}'.format(cur_key,cur_count)
            cur_key = key
            cur_count = int(count)

    # emit the last key
    print '{0}\t{1}'.format(cur_key,cur_count)


# ------------------------------ run -------------------------------- #
if __name__ == "__main__":
    sys.stderr.write("reporter:counter:Wordcount,Reducer,1\n")
    reducer_runner()



Overwriting reducer3.2.B.py


In [119]:
!chmod a+x mapper3.2.B.py
!chmod a+x reducer3.2.B.py

In [380]:
# Hadoop command

!hdfs dfs -rm -r /home/cloudera/W261/Spring2017/HW3/hw3.2.B-output

!sudo hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar \
    -mapper /home/cloudera/W261/Spring2017/HW3/mapper3.2.B.py \
    -reducer /home/cloudera/W261/Spring2017/HW3/reducer3.2.B.py \
    -input /home/cloudera/dfs/Consumer_Complaints/Consumer_Complaints.csv  \
    -output /home/cloudera/W261/Spring2017/HW3/hw3.2.B-output \
    -numReduceTasks 2


Deleted /home/cloudera/W261/Spring2017/HW3/hw3.2.B-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar] /tmp/streamjob1994643580797449871.jar tmpDir=null
17/01/29 02:14:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 02:14:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 02:14:41 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/29 02:14:41 INFO mapreduce.JobSubmitter: number of splits:2
17/01/29 02:14:41 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1484923209747_0228
17/01/29 02:14:41 INFO impl.YarnClientImpl: Submitted application application_1484923209747_0228
17/01/29 02:14:41 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1484923209747_0228/
17/01/29 02:14:41 INFO mapreduce.Job: Running job: job_1484923209747_0228
17/01/29 02:14:49 INFO mapreduce.Job: Job job_1484923209747_0228 running in uber mode : fa

### 3.2.B OUTPUT/ANSWER

The default split is two mappers and I suggested Hadoop to run two reducers as well.

In [381]:
!echo "----------------- part 0 --------------"
!hdfs dfs -cat /home/cloudera/W261/Spring2017/HW3/hw3.2.B-output/part-00000 | head  -5
!echo "----------------- part 1-------------- "
!hdfs dfs -cat /home/cloudera/W261/Spring2017/HW3/hw3.2.B-output/part-00001 | head  -5

----------------- part 0 --------------
apr	3431
account	16555
applied	139
arbitration	168
bankruptcy	222
----------------- part 1-------------- 
atm	2422
advertising	1193
application	8868
balance	597
cancelling	2795


### INSERT SCREENSHOT OF JOB TRACKER UI COUNTERS

Only the upper part of the wordcounts are shown here but, in case of interest, the counters can also be seen above in Hadoop output.

![](counter-32B.png)

### 3.2.C SOLUTION

In [407]:
%%writefile mapper3.2.C.py
#!/usr/bin/env python


import sys 
import re
from csv import reader

# ------------------------------ define function ---------------------------- #
def mapper_runner():

    WORD_RE = re.compile(r"[\w']+")

    for row in reader(sys.stdin):
        try:
             # the values in the first (id) column are integers except the header
            is_header = int(row[0])
            # Issue is the fourth column
            issue = row[3]
            for word in WORD_RE.findall(issue):
                print '{0}\t1'.format(word)
        except:
            # if the id can't be cast to int, it is a header, do not map it
            pass

# -------------------------------- run ---------------------------------------- # 

if __name__ == "__main__":
    sys.stderr.write("reporter:counter:Wordcount,Mapper,1\n")
    mapper_runner()


Overwriting mapper3.2.C.py


In [406]:
%%writefile combiner3.2.C.py
#!/usr/bin/env python


import sys

# ------------------------------ define functions -------------------------------- #

def combiner_runner():
    
    # define variables
    cur_key = None
    cur_count = 0
    total = 0

    for line in sys.stdin:
        key,count = line.lower().strip().split()
        # start with total counts   
        if key == cur_key:
            cur_count += int(count)
        else:
            if cur_key:
                print '{0}\t{1}'.format(cur_key,cur_count)
            cur_key = key
            cur_count = int(count)

    # emit the last key
    print '{0}\t{1}'.format(cur_key,cur_count)


# ------------------------------ run -------------------------------- #
if __name__ == "__main__":
    sys.stderr.write("reporter:counter:Wordcount,Combiner,1\n")
    combiner_runner()



Overwriting combiner3.2.C.py


In [398]:
%%writefile reducer3.2.C.py
#!/usr/bin/env python

import sys

# ------------------------------ define functions -------------------------------- #

def reducer_runner():
    
    # define variables
    cur_key = None
    cur_count = 0

    for line in sys.stdin:
        key,count = line.lower().strip().split()
        if key == cur_key:
            cur_count += int(count)
        else:
            if cur_key:
                print '{0}\t{1}'.format(cur_key,cur_count)
            cur_key = key
            cur_count = int(count)

    # emit the last key
    print '{0}\t{1}'.format(cur_key,cur_count)


# ------------------------------ run -------------------------------- #
if __name__ == "__main__":
    sys.stderr.write("reporter:counter:Wordcount,Reducer,1\n")
    reducer_runner()



Overwriting reducer3.2.C.py


In [239]:
!chmod a+x mapper3.2.C.py
!chmod a+x combiner3.2.C.py
!chmod a+x reducer3.2.C.py

In [408]:
# Hadoop command


!hdfs dfs -rm -r /home/cloudera/W261/Spring2017/HW3/hw3.2.C-output

!sudo hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar \
    -mapper /home/cloudera/W261/Spring2017/HW3/mapper3.2.C.py \
    -combiner /home/cloudera/W261/Spring2017/HW3/combiner3.2.C.py \
    -reducer /home/cloudera/W261/Spring2017/HW3/reducer3.2.C.py \
    -input /home/cloudera/dfs/Consumer_Complaints/Consumer_Complaints.csv  \
    -output /home/cloudera/W261/Spring2017/HW3/hw3.2.C-output \
    -numReduceTasks 1


Deleted /home/cloudera/W261/Spring2017/HW3/hw3.2.C-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar] /tmp/streamjob6133939959012997495.jar tmpDir=null
17/01/29 02:49:04 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 02:49:04 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 02:49:04 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/29 02:49:05 INFO mapreduce.JobSubmitter: number of splits:2
17/01/29 02:49:05 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1484923209747_0232
17/01/29 02:49:05 INFO impl.YarnClientImpl: Submitted application application_1484923209747_0232
17/01/29 02:49:05 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1484923209747_0232/
17/01/29 02:49:05 INFO mapreduce.Job: Running job: job_1484923209747_0232
17/01/29 02:49:12 INFO mapreduce.Job: Job job_1484923209747_0232 running in uber mode : fa

### 3.2.C OUTPUT/ANSWER

By adding a combiner (which shares the same code as the reducer), the calculations were sped up. In the end, while Hadoop makes no guarantee that a combiner would be called, two combiners were invoked.

In [410]:
!hdfs dfs -cat /home/cloudera/W261/Spring2017/HW3/hw3.2.C-output/part-00000 | head -5


apr	3431
atm	1056
account	6753
advertising	500
application	2906


### INSERT SCREENSHOT OF JOB TRACKER UI COUNTERS

![combiner-counter](counter-32C.png)

In [71]:
%%writefile frequencies_mapper3.2.C.py
#!/usr/bin/env python
# START STUDENT CODE HW32CFREQMAPPER
import sys
import re
import os
from csv import reader

# ------------------------ define functions ----------------------- # 
def grouper(word,num_reducers):
    # I guess this wouldn't really matter as it would be sent to one reducer anyway but it is better to be explicit
    if num_reducers == 1:
        return 0
    elif num_reducers == 2:
        # first group: words starting with a-m
        if(word[0]<='m'):
            return 0
        # second group: words starting with n-z
        else:
            return 1
    else:
        raise ValueError("Grouping for only 1 and 2 reducers is supported.")

def mapper_runner(num_reducer):
    WORD_RE = re.compile(r"[\w']+")
    for row in reader(sys.stdin):
        try:
            # the values in the first (id) column are integers except the header
            is_header = int(row[0])
            issue = row[3]
            for word in WORD_RE.findall(issue.lower()): 
                print '{0}\t{1}\t{2}'.format(grouper(word,num_reducer), word, 1)
                print '{0}\t{1}\t{2}'.format(grouper(word,num_reducer), '*', 1)
        except:
            # if can't be cast to int, it is a header, do not map it
            pass

# ----------------------------- run ------------------------------- #

if __name__ == "__main__":
    numReducers = int(os.environ.get('NUM_PARTITIONS', '1')) #default to one reducer
    mapper_runner(numReducers)
# END STUDENT CODE HW32CFREQMAPPER

Overwriting frequencies_mapper3.2.C.py


In [430]:
%%writefile frequencies_reducer3.2.C.py
#!/usr/bin/env python
# START STUDENT CODE HW32CFREQREDUCER

import sys

# -------------------------- define function ------------------------ #

def reducer_runner():
    # initialize
    cur_key = None
    cur_count = 0
    total = 0
    # loop over the rows 
    for line in sys.stdin:
        group,key,count = line.lower().strip().split()
        # start with total counts 
        if key == '*':
            total += int(count)
        else: 
            if key == cur_key:
                cur_count += int(count)
            else:
                if cur_key:
                    rel_freq = float(cur_count)/total
                    print '{0}\t{1}\t{2}'.format(cur_key,cur_count,rel_freq)
                cur_key = key
                cur_count = int(count)
    # emit the last key
    rel_freq = float(cur_count)/total
    print '{0}\t{1}\t{2}'.format(cur_key,cur_count,rel_freq)

# ---------------------------- run -------------------------------- #

if __name__ == "__main__":
    reducer_runner()

# END STUDENT CODE HW32CFREQREDUCER

Overwriting frequencies_reducer3.2.C.py


In [420]:
!chmod a+x frequencies_mapper3.2.C.py
!chmod a+x frequencies_reducer3.2.C.py

In [431]:
# Hadoop command
# START STUDENT CODE HW32CFREQHADOOP

!hdfs dfs -rm -r /home/cloudera/W261/Spring2017/HW3/frequencies3.2.C-output 

!hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.partition.keycomparator.options=-k2,2 \
    -mapper /home/cloudera/W261/Spring2017/HW3/frequencies_mapper3.2.C.py \
    -reducer /home/cloudera/W261/Spring2017/HW3/frequencies_reducer3.2.C.py \
    -input  /home/cloudera/dfs/Consumer_Complaints/Consumer_Complaints.csv \
    -output /home/cloudera/W261/Spring2017/HW3/frequencies3.2.C-output \
    -numReduceTasks 1 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner 
# END STUDENT CODE HW32CFREQHADOOP

Deleted /home/cloudera/W261/Spring2017/HW3/frequencies3.2.C-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar] /tmp/streamjob5343069697529019786.jar tmpDir=null
17/01/29 03:07:47 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 03:07:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 03:07:48 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/29 03:07:48 INFO mapreduce.JobSubmitter: number of splits:2
17/01/29 03:07:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1484923209747_0235
17/01/29 03:07:49 INFO impl.YarnClientImpl: Submitted application application_1484923209747_0235
17/01/29 03:07:49 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1484923209747_0235/
17/01/29 03:07:49 INFO mapreduce.Job: Running job: job_1484923209747_0235
17/01/29 03:07:56 INFO mapreduce.Job: Job job_1484923209747_0235 running in uber 

### 3.2.C OUTPUT/ANSWER

Using a single reducer: 
- What are the top 50 most frequent terms in your word count analysis?    
- Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order.    
- Present bottom 10 tokens (least frequent items). 

In [442]:
!echo "------------------------ top 50 -----------------------"
!hdfs dfs -cat /home/cloudera/W261/Spring2017/HW3/frequencies3.2.C-output/part-00000 | sort -k2,2nr | head -50 
!echo "------------------------ bottom 10 -----------------------"
!hdfs dfs -cat /home/cloudera/W261/Spring2017/HW3/frequencies3.2.C-output/part-00000 | sort -k2,2n | head -10 


------------------------ top 50 -----------------------
loan	119630	0.0887260180908
collection	72394	0.0536924797598
foreclosure	70487	0.0522781144961
modification	70487	0.0522781144961
account	57448	0.0426074754433
credit	55251	0.0409780257923
or	40508	0.0300435805469
payments	39993	0.0296616203419
escrow	36767	0.0272689919514
servicing	36767	0.0272689919514
report	34903	0.0258865185106
incorrect	29133	0.0216070808747
information	29069	0.0215596139754
on	29069	0.0215596139754
debt	27874	0.0206733179659
closing	19000	0.0140917357162
not	18477	0.0137038421488
attempts	17972	0.0133292986469
collect	17972	0.0133292986469
cont'd	17972	0.0133292986469
owed	17972	0.0133292986469
and	16448	0.0121989931084
management	16205	0.0120187672253
opening	16205	0.0120187672253
of	13983	0.0103707758168
my	10731	0.00795886399843
deposits	10555	0.00782833002548
withdrawals	10555	0.00782833002548
problems	9484	0.00703400113327
application	8868	0.00657713222795
communication	8671	0.00643102317868
tactics	86


### 3.2.1  
Using **2 reducers**: What are the top **50 most frequent terms** in your word count analysis? 


In [460]:
%%writefile frequencies_combiner3.2.1.py
#!/usr/bin/env python


import sys

# -------------------------- define function ------------------------ #

def combiner_runner():
    # initialize
    cur_key = None
    cur_count = 0
    # loop over the rows 
    for line in sys.stdin:
        group,key,count = line.lower().strip().split()
        # start with total counts 
        if key == cur_key:
                cur_count += int(count)
        else:
            if cur_key:
                print '{0}\t{1}\t{2}'.format(group,cur_key,cur_count)
            cur_key = key
            cur_count = int(count)
    # emit the last key
    print '{0}\t{1}\t{2}'.format(group,cur_key,cur_count)

# ---------------------------- run -------------------------------- #

if __name__ == "__main__":
    combiner_runner()


Overwriting frequencies_combiner3.2.1.py


In [446]:
!chmod a+x frequencies_mapper3.2.1.py
!chmod a+x frequencies_combiner3.2.1.py

In [77]:
# Hadoop command


!hdfs dfs -rm -r /home/cloudera/W261/Spring2017/HW3/frequencies3.2.1-output 

!hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.partition.keycomparator.options=-k2,2 \
    -mapper /home/cloudera/W261/Spring2017/HW3/frequencies_mapper3.2.C.py \
    -combiner /home/cloudera/W261/Spring2017/HW3/frequencies_combiner3.2.1.py \
    -reducer /home/cloudera/W261/Spring2017/HW3/frequencies_reducer3.2.C.py \
    -input  /home/cloudera/dfs/Consumer_Complaints/Consumer_Complaints.csv \
    -output /home/cloudera/W261/Spring2017/HW3/frequencies3.2.1-output \
    -numReduceTasks 2 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -cmdenv NUM_PARTITIONS=2


Deleted /home/cloudera/W261/Spring2017/HW3/frequencies3.2.1-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar] /tmp/streamjob4224400894549206311.jar tmpDir=null
17/01/31 02:04:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/31 02:04:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/31 02:04:05 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/31 02:04:05 INFO mapreduce.JobSubmitter: number of splits:2
17/01/31 02:04:06 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1484923209747_0374
17/01/31 02:04:06 INFO impl.YarnClientImpl: Submitted application application_1484923209747_0374
17/01/31 02:04:06 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1484923209747_0374/
17/01/31 02:04:06 INFO mapreduce.Job: Running job: job_1484923209747_0374
17/01/31 02:04:13 INFO mapreduce.Job: Job job_1484923209747_0374 running in uber 

In [78]:
!hdfs dfs -ls /home/cloudera/W261/Spring2017/HW3/frequencies3.2.1-output

Found 3 items
-rw-r--r--   1 root supergroup          0 2017-01-31 02:04 /home/cloudera/W261/Spring2017/HW3/frequencies3.2.1-output/_SUCCESS
-rw-r--r--   1 root supergroup       5685 2017-01-31 02:04 /home/cloudera/W261/Spring2017/HW3/frequencies3.2.1-output/part-00000
-rw-r--r--   1 root supergroup       3812 2017-01-31 02:04 /home/cloudera/W261/Spring2017/HW3/frequencies3.2.1-output/part-00001


In [79]:
!echo "------------------------ top 50 -----------------------"
!hdfs dfs -cat /home/cloudera/W261/Spring2017/HW3/frequencies3.2.C-output/part-0000* | sort -k2,2nr | head -50 
!echo "------------------------ bottom 10 -----------------------"
!hdfs dfs -cat /home/cloudera/W261/Spring2017/HW3/frequencies3.2.C-output/part-0000* | sort -k2,2n | head -10 


------------------------ top 50 -----------------------
loan	119630	0.0887260180908
collection	72394	0.0536924797598
foreclosure	70487	0.0522781144961
modification	70487	0.0522781144961
account	57448	0.0426074754433
credit	55251	0.0409780257923
or	40508	0.0300435805469
payments	39993	0.0296616203419
escrow	36767	0.0272689919514
servicing	36767	0.0272689919514
report	34903	0.0258865185106
incorrect	29133	0.0216070808747
information	29069	0.0215596139754
on	29069	0.0215596139754
debt	27874	0.0206733179659
closing	19000	0.0140917357162
not	18477	0.0137038421488
attempts	17972	0.0133292986469
collect	17972	0.0133292986469
cont'd	17972	0.0133292986469
owed	17972	0.0133292986469
and	16448	0.0121989931084
management	16205	0.0120187672253
opening	16205	0.0120187672253
of	13983	0.0103707758168
my	10731	0.00795886399843
deposits	10555	0.00782833002548
withdrawals	10555	0.00782833002548
problems	9484	0.00703400113327
application	8868	0.00657713222795
communication	8671	0.00643102317868
tactics	86


## Shopping Cart Analysis



In [477]:
# Download data
!wget https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0 -O ProductPurchaseData.csv
!hdfs dfs -mkdir /home/cloudera/dfs/ProductPurchaseData
!hdfs dfs -put ProductPurchaseData.csv /home/cloudera/dfs/ProductPurchaseData/ProductPurchaseData.csv

--2017-01-29 04:25:09--  https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0
Resolving www.dropbox.com... 162.125.65.1
Connecting to www.dropbox.com|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/UY9M2LxUAwTEt8EojvRYXbLengRuHknPH3cS7pkQ9m8Zc59uNU5h0JKyheEiDw7f/file [following]
--2017-01-29 04:25:09--  https://dl.dropboxusercontent.com/content_link/UY9M2LxUAwTEt8EojvRYXbLengRuHknPH3cS7pkQ9m8Zc59uNU5h0JKyheEiDw7f/file
Resolving dl.dropboxusercontent.com... 162.125.65.6
Connecting to dl.dropboxusercontent.com|162.125.65.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3458517 (3.3M) [text/plain]
Saving to: “ProductPurchaseData.csv”

100%[======================================>] 3,458,517   3.13M/s   in 1.1s    

2017-01-29 04:25:11 (3.13 MB/s) - “ProductPurchaseData.csv” saved [3458517/3458517]



In [669]:
%%writefile frequencies_mapper3.3.py
#!/usr/bin/env python

import sys
import os
from csv import reader

# ------------------------ define functions ----------------------- # 
def grouper(word,num_reducers):
    # I guess this wouldn't really matter as it would be sent to one reducer anyway but it is better to be explicit
    if num_reducers == 1:
        return 0
    elif num_reducers == 2:
        # first group: words starting with a-m
        if(word[0]<='M'):
            return 0
        # second group: words starting with n-z
        else:
            return 1
    else:
        raise ValueError("Grouping for only 1 and 2 reducers is supported.")

        
def mapper_runner(num_reducer):
    for customer in reader(sys.stdin):
        products = customer[0].split()
        for product in products: 
            print '{0}\t{1}\t{2}'.format(grouper(product,num_reducer), product, 1)
            print '{0}\t{1}\t{2}'.format(grouper(product,num_reducer), '*', 1)

# ----------------------------- run ------------------------------- #

if __name__ == "__main__":
    numReducers = int(os.environ.get('NUM_PARTITIONS', '1')) #default to one reducer
    mapper_runner(numReducers)




Overwriting frequencies_mapper3.3.py


In [619]:
%%writefile frequencies_reducer3.3.py
#!/usr/bin/env python


import sys

# -------------------------- define function ------------------------ #

def reducer_runner():
    # initialize
    cur_key = None
    cur_count = 0
    total = 0
    # loop over the rows 
    for line in sys.stdin:
        group,key,count = line.strip().split('\t',3)
        # start with total counts 
        if key == '*':
            total += int(count)
        else: 
            if key == cur_key:
                cur_count += int(count)
            else:
                if cur_key:
                    rel_freq = float(cur_count)/total
                    print '{0}\t{1}\t{2}'.format(cur_key,cur_count,rel_freq)
                cur_key = key
                cur_count = int(count)
    # emit the last key
    rel_freq = float(cur_count)/total
    print '{0}\t{1}\t{2}'.format(cur_key,cur_count,rel_freq)

# ---------------------------- run -------------------------------- #

if __name__ == "__main__":
    reducer_runner()


Overwriting frequencies_reducer3.3.py


In [472]:
!chmod a+x frequencies_*.py

In [642]:
# Hadoop command


!hdfs dfs -rm -r /home/cloudera/W261/Spring2017/HW3/frequencies3.3-output 

!hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D stream.num.map.output.key.fields=3 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.partition.keycomparator.options=-k2,2 \
    -mapper /home/cloudera/W261/Spring2017/HW3/frequencies_mapper3.3.py \
    -reducer /home/cloudera/W261/Spring2017/HW3/frequencies_reducer3.3.py \
    -input  /home/cloudera/dfs/ProductPurchaseData/ProductPurchaseData.csv \
    -output /home/cloudera/W261/Spring2017/HW3/frequencies3.3-output \
    -numReduceTasks 1 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -cmdenv NUM_PARTITIONS=1


Deleted /home/cloudera/W261/Spring2017/HW3/frequencies3.3-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar] /tmp/streamjob5100245058245541883.jar tmpDir=null
17/01/29 14:45:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 14:45:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 14:45:23 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/29 14:45:23 INFO mapreduce.JobSubmitter: number of splits:2
17/01/29 14:45:23 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1484923209747_0277
17/01/29 14:45:24 INFO impl.YarnClientImpl: Submitted application application_1484923209747_0277
17/01/29 14:45:24 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1484923209747_0277/
17/01/29 14:45:24 INFO mapreduce.Job: Running job: job_1484923209747_0277
17/01/29 14:45:31 INFO mapreduce.Job: Job job_1484923209747_0277 running in uber mo

In [643]:
!echo '#----------------- number of unique items --------------------------- #'
!hdfs dfs -cat /home/cloudera/W261/Spring2017/HW3/frequencies3.3-output/part-00000 | wc -l
!echo '#-------------------- top 50 highest frequency ---------------------- #'
!hdfs dfs -cat /home/cloudera/W261/Spring2017/HW3/frequencies3.3-output/part-00000 | sort -k2,2nr | head -50

#----------------- number of unique items --------------------------- #
12592
#-------------------- top 50 highest frequency ---------------------- #
DAI62779	6667	0.0175067747831
FRO40251	3881	0.010191059387
ELE17451	3875	0.0101753040775
GRO73461	3602	0.00945843749344
SNA80324	3044	0.00799319370628
ELE32164	2851	0.0074863979161
DAI75645	2736	0.00718442114993
SNA45677	2455	0.0064465474865
FRO31317	2330	0.0061183118711
DAI85309	2293	0.00602115412894
ELE26917	2292	0.00601852824402
FRO80039	2233	0.00586360103355
GRO21487	2115	0.00555374661261
SNA99873	2083	0.00546971829507
GRO59710	2004	0.00526227338613
GRO71621	1920	0.00504169905258
FRO85978	1918	0.00503644728273
GRO30386	1840	0.00483162825872
ELE74009	1816	0.00476860702057
GRO56726	1784	0.00468457870302
DAI63921	1773	0.00465569396887
GRO46854	1756	0.00461105392517
ELE66600	1713	0.00449814087347
DAI83733	1712	0.00449551498855
FRO32293	1702	0.00446925613932
ELE66810	1697	0.0044561267147
SNA55762	1646	0.00432220658362
DAI22177	1627	0.00427

In [622]:
%%writefile basket_mapper3.3.py
#!/usr/bin/env python

import sys
import re
import os
from csv import reader

# ------------------------ define functions ----------------------- # 
def grouper(word,num_reducers):
    # I guess this wouldn't really matter as it would be sent to one reducer anyway but it is better to be explicit
    if num_reducers == 1:
        return 0
    elif num_reducers == 2:
        # first group: words starting with a-m
        if(word[0]<='M'):
            return 0
        # second group: words starting with n-z
        else:
            return 1
    else:
        raise ValueError("Grouping for only 1 and 2 reducers is supported.")

        
def mapper_runner(num_reducer):
    max_basket_size = 0
    WORD_RE = re.compile(r"[\w']+")
    for customer in reader(sys.stdin):
        basket_size = len(WORD_RE.findall(customer[0]))
        if basket_size > max_basket_size:
            max_basket_size = basket_size
    print max_basket_size
            
        

# ----------------------------- run ------------------------------- #

if __name__ == "__main__":
    numReducers = int(os.environ.get('NUM_PARTITIONS', '1')) #default to one reducer
    mapper_runner(numReducers)



Overwriting basket_mapper3.3.py


In [590]:
!chmod a+x basket_mapper3.3.py

In [595]:
# Hadoop command


!hdfs dfs -rm -r /home/cloudera/W261/Spring2017/HW3/basket3.3-output 

!hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -mapper /home/cloudera/W261/Spring2017/HW3/basket_mapper3.3.py \
    -input  /home/cloudera/dfs/ProductPurchaseData/ProductPurchaseData.csv \
    -output /home/cloudera/W261/Spring2017/HW3/basket3.3-output \
    -numReduceTasks 0 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -cmdenv NUM_PARTITIONS=1


Deleted /home/cloudera/W261/Spring2017/HW3/basket3.3-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar] /tmp/streamjob5253531340811941659.jar tmpDir=null
17/01/29 13:04:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 13:04:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 13:04:23 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/29 13:04:23 INFO mapreduce.JobSubmitter: number of splits:2
17/01/29 13:04:23 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1484923209747_0259
17/01/29 13:04:23 INFO impl.YarnClientImpl: Submitted application application_1484923209747_0259
17/01/29 13:04:24 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1484923209747_0259/
17/01/29 13:04:24 INFO mapreduce.Job: Running job: job_1484923209747_0259
17/01/29 13:04:31 INFO mapreduce.Job: Job job_1484923209747_0259 running in uber mode : 

In [596]:
!echo "----------------- largest basket size -------------------- "
!hdfs dfs -cat /home/cloudera/W261/Spring2017/HW3/basket3.3-output/part-00000

----------------- largest basket size -------------------- 
37	



## Using 2 reducers


In [69]:
# Hadoop command


!hdfs dfs -rm -r /home/cloudera/W261/Spring2017/HW3/frequencies3.3.1-output 

!hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D stream.num.map.output.key.fields=3 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.partition.keycomparator.options=-k2,2 \
    -mapper /home/cloudera/W261/Spring2017/HW3/frequencies_mapper3.3.py \
    -reducer /home/cloudera/W261/Spring2017/HW3/frequencies_reducer3.3.py \
    -input  /home/cloudera/dfs/ProductPurchaseData/ProductPurchaseData.csv \
    -output /home/cloudera/W261/Spring2017/HW3/frequencies3.3.1-output \
    -numReduceTasks 2 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -cmdenv NUM_PARTITIONS=2


Deleted /home/cloudera/W261/Spring2017/HW3/frequencies3.3.1-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar] /tmp/streamjob5941898849205375092.jar tmpDir=null
17/01/31 01:53:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/31 01:53:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/31 01:53:18 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/31 01:53:18 INFO mapreduce.JobSubmitter: number of splits:2
17/01/31 01:53:18 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1484923209747_0372
17/01/31 01:53:18 INFO impl.YarnClientImpl: Submitted application application_1484923209747_0372
17/01/31 01:53:18 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1484923209747_0372/
17/01/31 01:53:18 INFO mapreduce.Job: Running job: job_1484923209747_0372
17/01/31 01:53:25 INFO mapreduce.Job: Job job_1484923209747_0372 running in uber 

In [70]:
!echo '#----------------- number of unique items --------------------------- #'
!hdfs dfs -cat /home/cloudera/W261/Spring2017/HW3/frequencies3.3.1-output/part-0000* | wc -l
!echo '#-------------------- top 50 highest frequency ---------------------- #'
!hdfs dfs -cat /home/cloudera/W261/Spring2017/HW3/frequencies3.3.1-output/part-0000* | sort -k2,2nr | head -50

#----------------- number of unique items --------------------------- #
12592
#-------------------- top 50 highest frequency ---------------------- #
DAI62779	6667	0.0216475095785
FRO40251	3881	0.0126014676278
ELE17451	3875	0.0125819858432
GRO73461	3602	0.0116955646471
SNA80324	3044	0.0417879303718
ELE32164	2851	0.00925709461653
DAI75645	2736	0.00888369374635
SNA45677	2455	0.0337021580364
FRO31317	2330	0.00756542632638
DAI85309	2293	0.00744528865511
ELE26917	2292	0.00744204169102
FRO80039	2233	0.00725047080979
GRO21487	2115	0.00686732904734
SNA99873	2083	0.0285953544561
GRO59710	2004	0.00650691603351
GRO71621	1920	0.00623417105007
FRO85978	1918	0.00622767712189
GRO30386	1840	0.00597441392298
ELE74009	1816	0.00589648678486
GRO56726	1784	0.00579258393402
DAI63921	1773	0.00575686732905
GRO46854	1756	0.00570166893954
ELE66600	1713	0.00556204948373
DAI83733	1712	0.00555880251964
FRO32293	1702	0.00552633287876
ELE66810	1697	0.00551009805832
SNA55762	1646	0.022596233046
DAI22177	1627	0.005282

Finding the largest basket does not require a reducer, hence the code and results are the same as before.


## Pairs

Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. 

Use the Pairs pattern (lecture 3)  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support



Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.

Step 1:

Calculate frequency of each unique pair. 

In [703]:
%%writefile pairs_mapper3.4.py
#!/usr/bin/env python

import sys
import os
from csv import reader

# ------------------------ define functions ----------------------- # 
def grouper(word,num_reducers):
    # I guess this wouldn't really matter as it would be sent to one reducer anyway but it is better to be explicit
    if num_reducers == 1:
        return 0
    elif num_reducers == 2:
        # first group: words starting with a-m
        if(word[0]<='M'):
            return 0
        # second group: words starting with n-z
        else:
            return 1
    else:
        raise ValueError("Grouping for only 1 and 2 reducers is supported.")
        
def get_pairs(item_list):
    for i, product in enumerate(item_list):
        for j in range(i+1, len(item_list)):
            yield [product, item_list[j]]

        
def mapper_runner(num_reducer):
    
    for customer in reader(sys.stdin):
        # do not take repetitions into account
        basket = customer[0].split()
        products = sorted(list(set(basket)))
        # get all the unique pairs,  
        pairs = list(get_pairs(products))
        for pair in pairs:
            print '{0}\t{1}\t{2}\t{3}'.format(grouper(pair[0],num_reducer),pair[0], pair[1],1) 
        # count total number of baskets    
        print '{0}\t{1}\t{2}\t{3}'.format(grouper('*',num_reducer),'*', '*',1)
       

    # ----------------------------- run ------------------------------- #

if __name__ == "__main__":
    sys.stderr.write("reporter:counter:Pairs analysis,Mapper,1\n")
    numReducers = int(os.environ.get('NUM_PARTITIONS', '1')) #default to one reducer
    mapper_runner(numReducers)




Overwriting pairs_mapper3.4.py


In [699]:
%%writefile pairs_reducer3.4.py
#!/usr/bin/env python


import sys

# -------------------------- define function ------------------------ #


def count_emitter(key,count):
    print '{0}\t{1}\t{2}'.format(key[0],key[1],count)

def reducer_runner():
    # initialize
    cur_key = None
    cur_count = 0
    # loop over the rows 
    for line in sys.stdin:
        group,pair0,pair1,count = line.strip().split('\t',4)
        key = [pair0,pair1]
        if key == cur_key:
            cur_count += int(count)
        else:
            if cur_key:
                count_emitter(cur_key,cur_count)
            cur_key = key
            cur_count = int(count)
    # emit the last key 
    count_emitter(cur_key,cur_count)

# ---------------------------- run -------------------------------- #

if __name__ == "__main__":
    sys.stderr.write("reporter:counter:Pairs analysis,Reducer,1\n")
    reducer_runner()



Overwriting pairs_reducer3.4.py


In [688]:
!chmod a+x *3.4.py

In [60]:
# Hadoop command


!hdfs dfs -rm -r /home/cloudera/W261/Spring2017/HW3/freq_pair.3.4-output 

!time hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D stream.num.map.output.key.fields=3 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.partition.keycomparator.options=-k2,2 \
    -mapper /home/cloudera/W261/Spring2017/HW3/pairs_mapper3.4.py \
    -reducer /home/cloudera/W261/Spring2017/HW3/pairs_reducer3.4.py \
    -input  /home/cloudera/dfs/ProductPurchaseData/ProductPurchaseData.csv \
    -output /home/cloudera/W261/Spring2017/HW3/freq_pair.3.4-output \
    -numReduceTasks 2 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -cmdenv NUM_PARTITIONS=2


Deleted /home/cloudera/W261/Spring2017/HW3/freq_pair.3.4-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar] /tmp/streamjob8769374255391815545.jar tmpDir=null
17/01/31 01:31:12 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/31 01:31:12 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/31 01:31:12 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/31 01:31:13 INFO mapreduce.JobSubmitter: number of splits:2
17/01/31 01:31:13 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1484923209747_0366
17/01/31 01:31:13 INFO impl.YarnClientImpl: Submitted application application_1484923209747_0366
17/01/31 01:31:13 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1484923209747_0366/
17/01/31 01:31:13 INFO mapreduce.Job: Running job: job_1484923209747_0366
17/01/31 01:31:19 INFO mapreduce.Job: Job job_1484923209747_0366 running in uber mod

In [61]:
!hdfs dfs -ls /home/cloudera/W261/Spring2017/HW3/freq_pair.3.4-output # not the best grouping!

Found 3 items
-rw-r--r--   1 root supergroup          0 2017-01-31 01:31 /home/cloudera/W261/Spring2017/HW3/freq_pair.3.4-output/_SUCCESS
-rw-r--r--   1 root supergroup   16871458 2017-01-31 01:31 /home/cloudera/W261/Spring2017/HW3/freq_pair.3.4-output/part-00000
-rw-r--r--   1 root supergroup     710187 2017-01-31 01:31 /home/cloudera/W261/Spring2017/HW3/freq_pair.3.4-output/part-00001


Step 2:
Sort and calculate relative frequency

In [8]:
%%writefile pairs_reducer3.41.py
#!/usr/bin/env python


import sys

# -------------------------- define function ------------------------ #


def count_emitter(key,count,total,s=100):
    if count > s:
        print '{0}\t{1}\t{2}\t{3}'.format(key[0],key[1],count,float(count)/total)
    else:
        pass

def reducer_runner():
    # initialize
    num_baskets = 0
    # loop over the rows 
    for basket in sys.stdin:
        pair0,pair1,count = basket.strip().split('\t',3)
        count = int(count)
        key = [pair0,pair1]
        if key == ['*','*']:
            total = int(count)
        else:
            count_emitter(key,count,total)

# ---------------------------- run -------------------------------- #

if __name__ == "__main__":
    sys.stderr.write("reporter:counter:Pairs analysis,Reducer 2,1\n")
    reducer_runner()


Overwriting pairs_reducer3.41.py


In [2]:
!chmod a+x pairs_reducer3.41.py

In [62]:
# Hadoop command

!hdfs dfs -rm -r /home/cloudera/W261/Spring2017/HW3/freq_pair.3.41-output 

!time hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D stream.num.map.output.key.fields=4 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.partition.keycomparator.options=-k3,3nr \
    -mapper 'cat' \
    -reducer /home/cloudera/W261/Spring2017/HW3/pairs_reducer3.41.py \
    -input  /home/cloudera/W261/Spring2017/HW3/freq_pair.3.4-output/part-0000* \
    -output /home/cloudera/W261/Spring2017/HW3/freq_pair.3.41-output \
    -numReduceTasks 1 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \


Deleted /home/cloudera/W261/Spring2017/HW3/freq_pair.3.41-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar] /tmp/streamjob2883337926454385287.jar tmpDir=null
17/01/31 01:32:21 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/31 01:32:21 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/31 01:32:21 INFO mapred.FileInputFormat: Total input paths to process : 2
17/01/31 01:32:22 INFO mapreduce.JobSubmitter: number of splits:3
17/01/31 01:32:22 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1484923209747_0367
17/01/31 01:32:22 INFO impl.YarnClientImpl: Submitted application application_1484923209747_0367
17/01/31 01:32:22 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1484923209747_0367/
17/01/31 01:32:22 INFO mapreduce.Job: Running job: job_1484923209747_0367
17/01/31 01:32:29 INFO mapreduce.Job: Job job_1484923209747_0367 running in uber mo

In [8]:
!hdfs dfs -cat /home/cloudera/W261/Spring2017/HW3/freq_pair.3.41-output/part-00000 | head -50

DAI62779	ELE17451	1592	0.0511880646925
FRO40251	SNA80324	1412	0.0454004694383
DAI75645	FRO40251	1254	0.0403202469374
FRO40251	GRO85051	1213	0.0390019613517
DAI62779	GRO73461	1139	0.0366226166361
DAI75645	SNA80324	1130	0.0363332368734
DAI62779	FRO40251	1070	0.0344040384554
DAI62779	SNA80324	923	0.0296775023311
DAI62779	DAI85309	918	0.0295167357963
ELE32164	GRO59710	911	0.0292916626475
DAI62779	DAI75645	882	0.0283592167454
FRO40251	GRO73461	882	0.0283592167454
DAI62779	ELE92920	877	0.0281984502106
FRO40251	FRO92469	835	0.026848011318
DAI62779	ELE32164	832	0.0267515513971
DAI75645	GRO73461	712	0.0228931545609
DAI43223	ELE32164	711	0.022861001254
DAI62779	GRO30386	709	0.02279669464
ELE17451	FRO40251	697	0.0224108549564
DAI85309	ELE99737	659	0.0211890292917
DAI62779	ELE26917	650	0.020899649529
GRO21487	GRO73461	631	0.0202887366966
DAI62779	SNA45677	604	0.0194205974084
ELE17451	SNA80324	597	0.0191955242597
DAI62779	GRO71621	595	0.0191312176457
DAI62779	SNA55762	593	0.0190669110318
DAI62779	D

Computational setup:
Single computer on Cloudera VM (CentOS 6), 2 cores and 8 GB RAM given to VM.

The first job that emitted the pairs and calculated their frequencies called 2 mappers and 2 reducers while the second job which calculated the relative frequencies and sorted the output called only one reducer and used only an identity mapper.

First job required 41 seconds, the second 35 seconds to run. 

<a name="3.5"></a>
##  Stripes
Repeat the pairs exercise using the stripes design pattern for finding cooccuring pairs.

Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs


In [30]:
%%writefile stripes_mapper35.py
#!/usr/bin/python


import sys
import os
from csv import reader
from collections import defaultdict
from itertools import repeat

# ------------------------ define functions ----------------------- # 
def grouper(word,num_reducers):
    # I guess this wouldn't really matter as it would be sent to one reducer anyway but it is better to be explicit
    if num_reducers == 1:
        return 0
    elif num_reducers == 2:
        # first group: words starting with a-m
        if(word[0]<='M'):
            return 0
        # second group: words starting with n-z
        else:
            return 1
    else:
        raise ValueError("Grouping for only 1 and 2 reducers is supported.")
        
def mapper_runner(num_reducer):
    
    for customer in reader(sys.stdin):
        # do not take repetitions into account
        basket = customer[0].split()
        products = sorted(list(set(basket)))
        stripe = {}
        # get all the unique pairs,  
        for i,product in enumerate(products):
            # generate stripe:
            #    1: get all of the other products 
            other_products = products[(i+1):]
            #    2: create a container for products and counts
            item = [None]*(len(other_products)*2)
            #    3: associate products with counts of 1
            item[::2] = other_products
            item[1::2] = repeat(1,len(other_products))
            item = dict(map(None, *[iter(item)]*2))
            #    4: place them in a dictionary to the product key
            stripe[product] = item
            print '[{0}-{1}]\t{1}\t{2}'.format(grouper(product,num_reducer),product,stripe[product]) 
            # stop the loop when only the last item remains in the basket
            if len(stripe[product]) == 1:
                break
        # count total number of baskets    
        print '[{0}-{1}]\t{1}\t{2}'.format(grouper('*',num_reducer),'*',{'*':1})
       

    # ----------------------------- run ------------------------------- #

if __name__ == "__main__":
    sys.stderr.write("reporter:counter:Stripes analysis,Mapper,1\n")
    numReducers = int(os.environ.get('NUM_PARTITIONS', '1')) #default to one reducer
    mapper_runner(numReducers)



Overwriting stripes_mapper35.py


In [54]:
%%writefile stripes_reducer35.py
#!/usr/bin/python


import os
import sys
import ast
import itertools
from collections import defaultdict

# ----------------------------- define functions --------------------- #

def add_dictionaries(dict1, dict2):
    new_dict = {}
    for k in itertools.chain(dict1.keys(), dict2.keys()):
        new_dict[k] = dict1.get(k, 0)+dict2.get(k, 0)
    return new_dict

def reducer_runner():
    # initialize
    cur_key = None
    cur_pair_dict = defaultdict(int)
    
    # loop over the rows 
    for line in sys.stdin:
        try:
            line = line.strip().split('\t',3)
            group,key,pair_dict = line
            # convert the print statement to dictionary
            pair_dict = ast.literal_eval(pair_dict)
            # merge and sum dictionaries if subsequent keys are the same
            if key == cur_key:
                cur_pair_dict = add_dictionaries(cur_pair_dict,pair_dict)  
            # if a new key comes, emit the current one and set the current key/dictionary to the new key/dictionary
            else:
                if cur_key:
                    print '{0}\t{1}'.format(cur_key,cur_pair_dict)
                cur_key = key
                cur_pair_dict = pair_dict 
        except ValueError:
            sys.stderr.write("reporter:counter:Stripes analysis,ValueError,1\n")
        # emit the last key 
    print '{0}\t{1}'.format(cur_key,cur_pair_dict)

# ---------------------------- run -------------------------------- #

if __name__ == "__main__":
    sys.stderr.write("reporter:counter:Stripes analysis,Reducer,1\n")
    reducer_runner()



Overwriting stripes_reducer35.py


In [80]:
!chmod a+x stripes_*

In [57]:
# Hadoop command


!hdfs dfs -rm -r /home/cloudera/W261/Spring2017/HW3/freq_stripes.3.5-output 

!time hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D stream.num.map.output.key.fields=3 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.partition.keycomparator.options=-k1,1 \
    -mapper /home/cloudera/W261/Spring2017/HW3/stripes_mapper35.py \
    -reducer /home/cloudera/W261/Spring2017/HW3/stripes_reducer35.py \
    -input   /home/cloudera/dfs/ProductPurchaseData/ProductPurchaseData.csv \
    -output /home/cloudera/W261/Spring2017/HW3/freq_stripes.3.5-output \
    -numReduceTasks 2 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -cmdenv NUM_PARTITIONS=2


Deleted /home/cloudera/W261/Spring2017/HW3/freq_stripes.3.5-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar] /tmp/streamjob2134867133890228098.jar tmpDir=null
17/01/31 01:24:21 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/31 01:24:21 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/31 01:24:21 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/31 01:24:21 INFO mapreduce.JobSubmitter: number of splits:2
17/01/31 01:24:22 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1484923209747_0364
17/01/31 01:24:22 INFO impl.YarnClientImpl: Submitted application application_1484923209747_0364
17/01/31 01:24:22 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1484923209747_0364/
17/01/31 01:24:22 INFO mapreduce.Job: Running job: job_1484923209747_0364
17/01/31 01:24:28 INFO mapreduce.Job: Job job_1484923209747_0364 running in uber 

In [35]:
%%writefile stripes_mapper351.py
#!/usr/bin/python

import sys
import os
import ast


for line in sys.stdin:
    line = line.strip().split('\t',2)
    key,pair_dict = line
    pairs = ast.literal_eval(pair_dict)
    # extract those pairs whose counts are over 100
    frequent_pairs = dict((pair, count) for pair, count in pairs.items() if count > 100)
    # and emit them
    for frequent_pair,count in frequent_pairs.items():
        print '{0}\t{1}\t{2}'.format(key,frequent_pair,count)

Overwriting stripes_mapper351.py


In [43]:
!chmod a+x stripes_mapper351.py

In [58]:
!hdfs dfs -rm -r /home/cloudera/W261/Spring2017/HW3/freq_stripes.3.51-output 

!time hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D stream.num.map.output.key.fields=3 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.partition.keycomparator.options=-k3,3nr \
    -mapper /home/cloudera/W261/Spring2017/HW3/stripes_mapper351.py \
    -reducer /home/cloudera/W261/Spring2017/HW3/pairs_reducer3.41.py \
    -input   /home/cloudera/W261/Spring2017/HW3/freq_stripes.3.5-output/* \
    -output /home/cloudera/W261/Spring2017/HW3/freq_stripes.3.51-output \
    -numReduceTasks 1 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \


Deleted /home/cloudera/W261/Spring2017/HW3/freq_stripes.3.51-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.9.1.jar] /tmp/streamjob4715335295154851299.jar tmpDir=null
17/01/31 01:25:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/31 01:25:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/31 01:25:59 INFO mapred.FileInputFormat: Total input paths to process : 2
17/01/31 01:25:59 WARN hdfs.DFSClient: Caught exception 
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1281)
	at java.lang.Thread.join(Thread.java:1355)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.closeResponder(DFSOutputStream.java:950)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.endBlock(DFSOutputStream.java:688)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.run(DFSOutputStream.java:877)
17/01/31 01:25:59 INFO mapreduce.JobSubmitter: number of 

In [59]:
!echo '# ---------------- top 50 frequent pairs --------------------------- #'
!hdfs dfs -cat /home/cloudera/W261/Spring2017/HW3/freq_stripes.3.51-output/part-00000 | head -50

# ---------------- top 50 frequent pairs --------------------------- #
DAI62779	ELE17451	1592	0.0511880646925
FRO40251	SNA80324	1412	0.0454004694383
DAI75645	FRO40251	1254	0.0403202469374
FRO40251	GRO85051	1213	0.0390019613517
DAI62779	GRO73461	1139	0.0366226166361
DAI75645	SNA80324	1130	0.0363332368734
DAI62779	FRO40251	1070	0.0344040384554
DAI62779	SNA80324	923	0.0296775023311
DAI62779	DAI85309	918	0.0295167357963
ELE32164	GRO59710	911	0.0292916626475
DAI62779	DAI75645	882	0.0283592167454
FRO40251	GRO73461	882	0.0283592167454
DAI62779	ELE92920	877	0.0281984502106
FRO40251	FRO92469	835	0.026848011318
DAI62779	ELE32164	832	0.0267515513971
DAI75645	GRO73461	712	0.0228931545609
DAI43223	ELE32164	711	0.022861001254
DAI62779	GRO30386	709	0.02279669464
ELE17451	FRO40251	697	0.0224108549564
DAI85309	ELE99737	659	0.0211890292917
DAI62779	ELE26917	650	0.020899649529
GRO21487	GRO73461	631	0.0202887366966
DAI62779	SNA45677	604	0.0194205974084
ELE17451	SNA80324	597	0.0191955242597
DAI62779	GRO716

Computational setup: single computer on Cloudera VM (CentOS 6) with two processors and 8 GB available to the VM, using two mappers and two reducers.

Stripes: 
First step which merged and summed the stripes took 85 seconds. (2 reducers)
Second step which calculated the relative frequencies and sorted the results took 27 seconds. (1 reducer)